# Imports

In [1]:
# Imports
import pandas as pd
import json
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from datasets import Dataset
from datasets import concatenate_datasets
from torch import random
import os

In [2]:
%load_ext autoreload
%autoreload 2

In [46]:
dataset1 = "ArtifactAI/arxiv-math-instruct-50k"
dataset3 = "zeroshot/arxiv-biology"
dataset4 = "camel-ai/biology"
dataset5 = "ayoubkirouane/arxiv-physics"
dataset6 = "jilp00/YouToks-Instruct-Quantum-Physics-III"
dataset7 = "jilp00/YouToks-Instruct-Quantum-Physics-II"
dataset8 = "mjphayes/machine_learning_questions"
dataset9 = "Mridul-Dixit/Machine-Learning-QA-Dataset"
dataset10 = "jilp00/YouToks-Instruct-Thermodynamics-of-Materials"
dataset11 = "whiteOUO/Ladder-machine-learning-QA"

In [47]:
# Load datasets
dataset1 = load_dataset(dataset1, split='train')

In [48]:
dataset1[0]

{'question': 'What structure is classified as a definite lie algebra?',
 'answer': 'A definite Lie algebra is a Lie algebra equipped with an inner product that is positive definite. Such an algebra is called a positive definite or a definite Lie algebra.'}

In [49]:
# concatenate what is in question and answer in one column (they are strings)
def concatenate_qa(line):
    line['text'] = line['question'] + "\n" + line['answer']
    return line


# apply the function to the dataset
dataset1 = dataset1.map(concatenate_qa)

In [50]:
# Drop question and answer columns
dataset1 = dataset1.remove_columns(['question', 'answer'])

In [51]:
dataset3 = load_dataset(dataset3, split='train')

In [52]:
# concatenate what is in title and abstract in one column (they are strings)
def concatenate_title_abstract(line):
    line['text'] = line['title'] + "\n" + line['abstract']
    return line

dataset3 = dataset3.map(concatenate_title_abstract)

In [53]:
# Drop title and abstract columns
dataset3 = dataset3.remove_columns(['title', 'abstract','id'])

In [54]:
dataset4 = load_dataset(dataset4, split='train')

In [55]:
# concatenate what is in message_1 and message_2 in one column (they are strings)
def concatenate_messages(line):
    line['text'] = line['message_1'] + "\n" + line['message_2']
    return line

dataset4 = dataset4.map(concatenate_messages)

In [56]:
# remove columns message_1 and message_2, role_1, topic
dataset4 = dataset4.remove_columns(['message_1', 'message_2', 'role_1', 'topic;'])

In [57]:
# remove sub_topic column
dataset4 = dataset4.remove_columns('sub_topic')

In [58]:
dataset5 = load_dataset(dataset5, split='train')

In [59]:
# concatenate what is in question and answer in one column (they are strings) with already existing function
dataset5 = dataset5.map(concatenate_qa)

In [60]:
# Drop question and answer columns
dataset5 = dataset5.remove_columns(['question', 'answer'])

In [61]:
dataset6 = load_dataset(dataset6, split='train')

In [62]:
# concatenate what is in question and response in one column (they are strings)
def concatenate_qr(line):
    line['text'] = line['question'] + "\n" + line['response']
    return line

dataset6 = dataset6.map(concatenate_qr)

In [63]:
# Drop question and response columns and system_prompt, token_count
dataset6 = dataset6.remove_columns(['question', 'response', 'system_prompt', 'token_count'])

In [64]:
dataset7 = load_dataset(dataset7, split='train')

In [65]:
# concatenate what is in question and response in one column (they are strings)
dataset7 = dataset7.map(concatenate_qr)

In [66]:
# Drop question and response columns and system_prompt, token_count,__index_level_0__
dataset7 = dataset7.remove_columns(['question', 'response', 'system_prompt', 'token_count','__index_level_0__'])

In [67]:
dataset8 = load_dataset(dataset8, split='train')

In [68]:
# concatenate what is in question and answer in one column (they are strings)
dataset8 = dataset8.map(concatenate_qa)

In [69]:
# Drop question and answer columns, __index_level_0__
dataset8 = dataset8.remove_columns(['question', 'answer','__index_level_0__'])

In [70]:
dataset9 = load_dataset(dataset9, split='train')

In [71]:
# concatenate what is in question and answer in one column (they are strings)
dataset9 = dataset9.map(concatenate_qa)

In [72]:
# Drop question and answer columns
dataset9 = dataset9.remove_columns(['question', 'answer'])

In [73]:
dataset10 = load_dataset(dataset10, split='train')

In [74]:
def filter_out_584(example, idx):
    return idx != 584

dataset10 = dataset10.filter(filter_out_584, with_indices=True)

In [75]:
# Only take the column response which contains string but take only what is after "\nQUESTION:" if it is not empty
def take_response(line):
    split = line['response'].split("\nQUESTION:")
    if split[1] != "":
        line['text'] = split[1]
    return line

dataset10 = dataset10.map(take_response)

In [76]:
# remove token_count, response
dataset10 = dataset10.remove_columns(['token_count', 'response'])

In [77]:
dataset11 = load_dataset(dataset11, split='train')

In [78]:
# concatenate what is in Question and Answer in one column (they are strings)
def concatenate_QA(line):
    line['text'] = line['Question'] + "\n" + line['Answer']
    return line

dataset11 = dataset11.map(concatenate_QA)

In [79]:
# Drop Question and Answer columns
dataset11 = dataset11.remove_columns(['Question', 'Answer'])

In [80]:

datasets = [dataset1, dataset3, dataset4, dataset5, dataset6, dataset7, dataset8, dataset9, dataset10, dataset11]

# Concatenate all datasets
datasets = concatenate_datasets(datasets)

In [81]:
# Save the dataset as a JSON file
datasets.to_json("datasets.json")

Creating json from Arrow format:   0%|          | 0/106 [00:00<?, ?ba/s]

121209893

In [82]:
file_path = 'datasets.json'

# Load the dataset
dataset = load_dataset('json', data_files=file_path)

Generating train split: 0 examples [00:00, ? examples/s]

In [45]:
from datasets import load_dataset
import torch
from transformers import DPRContextEncoder, DPRContextEncoderTokenizerFast

# Initialize the encoder and tokenizer
ctx_encoder = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')
ctx_tokenizer = DPRContextEncoderTokenizerFast.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')

def embed(batch):
    inputs = ctx_tokenizer(batch['text'], truncation=True, padding=True, return_tensors='pt')
    #inputs = {k: v.to('cuda') for k, v in inputs.items()}  # Move inputs to GPU if available
    with torch.no_grad():
        embeddings = ctx_encoder(**inputs).pooler_output
    return {'embeddings': embeddings.cpu().numpy()}

# Process the dataset in batches
dataset = dataset.map(embed, batched=True, batch_size=32)

# Save the dataset with embeddings
dataset.save_to_disk("encoded_dataset")

C:\Users\41794\anaconda3\envs\assMNLP\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identica

Map:   0%|          | 0/2992624 [00:00<?, ? examples/s]


KeyboardInterrupt

